### 4-Analytes Classification on the  preprocessed data using MLP
### Edgar Acuna
###  June 2021

In [33]:
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout
from keras.utils import to_categorical
import time

In [34]:
df1=pd.read_csv("c://onr2020/NRLset1_part1.csv",header=None)
df2=pd.read_csv("c://onr2020/NRLset1_part2.csv",header=None)
df3=pd.read_csv("c://onr2020/NRLset1_part3.csv",header=None)
df4=pd.read_csv("c://onr2020/NRLset1_part4.csv",header=None)
df5=pd.read_csv("c://onr2020/NRLset1_part5.csv",header=None)
df6=pd.read_csv("c://onr2020/NRLset1_part6.csv",header=None)
df7=pd.read_csv("c://onr2020/NRLset1_part7.csv",header=None)
df8=pd.read_csv("c://onr2020/NRLset1_part8.csv",header=None)
y=pd.read_csv("c://onr2020/labels.csv",header=None)
ys=pd.read_csv("c://onr2020/substrateIDs.csv",header=None)
subs=pd.read_csv("c://onr2020/substrates.csv",header=None)
dfset1=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8],ignore_index=True)
print('Size of the dataframe: {}'.format(dfset1.shape))

Size of the dataframe: (18000, 1701)


In [35]:
print('Size of the dataframe: {}'.format(subs.shape))

Size of the dataframe: (9, 1701)


In [36]:
dfset2=dfset1.copy()
dfset2['Analyte']=y
dfset2['substrate']=ys
dfsub1=dfset2[dfset2['substrate']==1]
dfsub2=dfset2[dfset2['substrate']==2]
dfsub3=dfset2[dfset2['substrate']==3]
dfsub4=dfset2[dfset2['substrate']==4]
dfsub5=dfset2[dfset2['substrate']==5]
dfsub6=dfset2[dfset2['substrate']==6]
dfsub7=dfset2[dfset2['substrate']==7]
dfsub8=dfset2[dfset2['substrate']==8]
dfsub9=dfset2[dfset2['substrate']==9]

In [37]:
mdfsub1=dfsub1.iloc[:,0:1701]
mdfsub2=dfsub2.iloc[:,0:1701]
mdfsub3=dfsub3.iloc[:,0:1701]
mdfsub4=dfsub4.iloc[:,0:1701]
mdfsub5=dfsub5.iloc[:,0:1701]
mdfsub6=dfsub6.iloc[:,0:1701]
mdfsub7=dfsub7.iloc[:,0:1701]
mdfsub8=dfsub8.iloc[:,0:1701]
mdfsub9=dfsub9.iloc[:,0:1701]

In [38]:
y1=subs.loc[0,:]
y2=subs.loc[1,:]
y3=subs.loc[2,:]
y4=subs.loc[3,:]
y5=subs.loc[4,:]
y6=subs.loc[5,:]
y7=subs.loc[6,:]
y8=subs.loc[7,:]
y9=subs.loc[8,:]

###  Centering

In [39]:
modsub1=mdfsub1.apply(lambda x : x -(np.sum(np.array(x)*np.array(y1))/np.sum(np.array(y1)*np.array(y1)))*y1,axis=1)
modsub2=mdfsub2.apply(lambda x : x -(np.sum(np.array(x)*np.array(y2))/np.sum(np.array(y2)*np.array(y2)))*y2,axis=1)
modsub3=mdfsub3.apply(lambda x : x -(np.sum(np.array(x)*np.array(y3))/np.sum(np.array(y3)*np.array(y3)))*y3,axis=1)
modsub4=mdfsub4.apply(lambda x : x -(np.sum(np.array(x)*np.array(y4))/np.sum(np.array(y4)*np.array(y4)))*y4,axis=1)
modsub5=mdfsub5.apply(lambda x : x -(np.sum(np.array(x)*np.array(y5))/np.sum(np.array(y5)*np.array(y5)))*y5,axis=1)
modsub6=mdfsub6.apply(lambda x : x -(np.sum(np.array(x)*np.array(y6))/np.sum(np.array(y6)*np.array(y6)))*y6,axis=1)
modsub7=mdfsub7.apply(lambda x : x -(np.sum(np.array(x)*np.array(y7))/np.sum(np.array(y7)*np.array(y7)))*y7,axis=1)
modsub8=mdfsub8.apply(lambda x : x -(np.sum(np.array(x)*np.array(y8))/np.sum(np.array(y8)*np.array(y8)))*y8,axis=1)
modsub9=mdfsub9.apply(lambda x : x -(np.sum(np.array(x)*np.array(y9))/np.sum(np.array(y9)*np.array(y9)))*y9,axis=1)

In [40]:
subdf=[modsub1,modsub2,modsub3,modsub4,modsub5,modsub6,modsub7,modsub8,modsub9]
cent_subs=pd.concat(subdf)
cent_subs.head()

,0,1,2,3,4,5,6,7,8,9,...,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700
4,0.002296,0.002603,0.002368,0.003507,0.003637,0.003537,0.004850,0.005059,0.004744,0.006244,...,-0.001858,-0.002531,-0.002143,-0.004422,-0.003123,-0.000884,-0.001225,-0.001406,-0.002792,-0.002432
5,-0.005471,0.003865,0.000351,-0.000060,-0.000258,-0.000582,0.002557,0.003072,0.000961,0.003941,...,-0.004245,-0.001915,-0.005149,-0.001717,-0.003814,-0.003100,-0.000355,-0.000540,-0.005481,-0.000857
8,-0.002496,0.016867,0.014204,0.005327,0.003649,0.006143,-0.015160,0.004201,0.003791,0.004599,...,0.002574,-0.018040,-0.018488,-0.006529,0.009548,0.004694,0.000399,0.018158,0.016369,0.007582
28,0.001313,0.001443,0.001536,0.001612,0.001687,0.001768,0.001866,0.001979,0.002079,0.002155,...,-0.004576,-0.004550,-0.004524,-0.004502,-0.004476,-0.004446,-0.004411,-0.004373,-0.004332,-0.004301
46,-0.001097,0.000040,0.001125,0.002056,0.001685,0.002006,0.000228,0.001179,0.001835,0.002005,...,-0.004655,-0.006438,-0.005102,-0.005563,-0.004804,-0.005517,-0.004411,-0.005738,-0.005031,-0.005584


### Normalization

In [41]:
b=cent_subs.iloc[:,0:1701]
b.head()

,0,1,2,3,4,5,6,7,8,9,...,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700
4,0.002296,0.002603,0.002368,0.003507,0.003637,0.003537,0.004850,0.005059,0.004744,0.006244,...,-0.001858,-0.002531,-0.002143,-0.004422,-0.003123,-0.000884,-0.001225,-0.001406,-0.002792,-0.002432
5,-0.005471,0.003865,0.000351,-0.000060,-0.000258,-0.000582,0.002557,0.003072,0.000961,0.003941,...,-0.004245,-0.001915,-0.005149,-0.001717,-0.003814,-0.003100,-0.000355,-0.000540,-0.005481,-0.000857
8,-0.002496,0.016867,0.014204,0.005327,0.003649,0.006143,-0.015160,0.004201,0.003791,0.004599,...,0.002574,-0.018040,-0.018488,-0.006529,0.009548,0.004694,0.000399,0.018158,0.016369,0.007582
28,0.001313,0.001443,0.001536,0.001612,0.001687,0.001768,0.001866,0.001979,0.002079,0.002155,...,-0.004576,-0.004550,-0.004524,-0.004502,-0.004476,-0.004446,-0.004411,-0.004373,-0.004332,-0.004301
46,-0.001097,0.000040,0.001125,0.002056,0.001685,0.002006,0.000228,0.001179,0.001835,0.002005,...,-0.004655,-0.006438,-0.005102,-0.005563,-0.004804,-0.005517,-0.004411,-0.005738,-0.005031,-0.005584


In [42]:
#normalize(b)
b1=b.apply(lambda x: x/(x**2).sum()**.5, axis=1)
b1.shape

(18000, 1701)

In [43]:
a1=np.array(dfsub1['Analyte'].values.tolist())
a2=np.array(dfsub2['Analyte'].values.tolist())
a3=np.array(dfsub3['Analyte'].values.tolist())
a4=np.array(dfsub4['Analyte'].values.tolist())
a5=np.array(dfsub5['Analyte'].values.tolist())
a6=np.array(dfsub6['Analyte'].values.tolist())
a7=np.array(dfsub7['Analyte'].values.tolist())
a8=np.array(dfsub8['Analyte'].values.tolist())
a9=np.array(dfsub9['Analyte'].values.tolist())
y=np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9),axis=None)
#print(len(y))

In [44]:
# Configuration options
feature_vector_length = 1701
num_classes = 18000
X=b1.iloc[:,0:1701]
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2,random_state=0)
# Convert target classes to categorical ones
ytrain=Y_train
ytest=Y_test
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)
print('Train dimension:')
print(X_train.shape)
print('Test dimension:')
print(X_test.shape)

Train dimension:
(14400, 1701)
Test dimension:
(3600, 1701)


In [53]:
# reshaping
print(X_train.shape[1])            
X_train=np.reshape(X_train,(14400,1,1701))
#X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[0]))

1701


ValueError: Must pass 2-d input. shape=(14400, 1, 1701)

In [ ]:
# Set the input shape
#input_shape = (feature_vector_length,)
#print(f'Feature shape: {input_shape}')

model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(300,1701)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(40, activation='softmax'))

model.summary()


In [32]:
# Configure the model and start training
import time
start_time = time.time()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
baseline_history=model.fit(X_train, Y_train, epochs=100, batch_size=150, verbose=1, validation_split=0.2)

Epoch 1/100


ValueError: in user code:

    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step

In [ ]:
# Test the model after training
test_results = model.evaluate(X_test, Y_test, verbose=1)
print(test_results)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')
print("--- %s seconds ---" % (time.time() - start_time))

In [16]:
#Computing F1-score
train_features = np.array(X_train)
test_features = np.array(X_test)
train_labels=np.array(Y_train)
test_labels=np.array(Y_test)
train_predictions_baseline = model.predict_classes(train_features, batch_size=150)
f1_train=sklearn.metrics.f1_score(ytrain, train_predictions_baseline, average="weighted")
test_predictions_baseline = model.predict_classes(test_features, batch_size=150)
f1_test=sklearn.metrics.f1_score(ytest, test_predictions_baseline, average="weighted")
print('f1_scores in training set',f1_train,'f1_scores in testing set',f1_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
f1_scores in training set 0.9903507362154318 f1_scores in testing set 0.9475780810594095


In [17]:
num_classes = 18000
#labels1=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
labels1=list(range(10,11))
#labels1=[3,26]
import time
start_time=time.time()
for j in labels1:
    yclass=y.copy()
    yclass[yclass!=j]=0 
    yclass[yclass==j]=1
    X_train, X_test, yclass_train, yclass_test = train_test_split(X,yclass,test_size=0.2,random_state=0)
    #print('Train dimension:');print(X_train.shape)
    #print('Test dimension:');print(X_test.shape)
    Y_train = to_categorical(yclass_train, num_classes)
    Y_test = to_categorical(yclass_test, num_classes)
    # Set the input shape
    input_shape = (feature_vector_length,) 
    #print(f'Feature shape: {input_shape}')
    # Create the model
    model = Sequential()
    model.add(Dropout(0.3, input_shape=input_shape))
    model.add(Dense(300, input_shape=input_shape, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    # Configure the model and start training
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.BinaryAccuracy(name="accuracy"), keras.metrics.Precision(name="precision"),keras.metrics.Recall(name="recall")])
    baseline_history=model.fit(X_train, Y_train, epochs=200, batch_size=150, validation_split=0.2)
    # Test the model after training
    test_results = model.evaluate(X_test, Y_test, verbose=1)
    #print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}% -Precision: {test_results[2]}% -Recall: {test_results[3]}%')
    #Computing F1-score
    train_features = np.array(X_train)
    test_features = np.array(X_test)
    train_labels=np.array(yclass_train)
    test_labels=np.array(yclass_test)
    train_predictions_baseline = model.predict_classes(train_features, batch_size=100)
    f1_train=sklearn.metrics.f1_score(train_labels, train_predictions_baseline, average=None)
    test_predictions_baseline = model.predict_classes(test_features, batch_size=150)
    f1_test=sklearn.metrics.f1_score(test_labels, test_predictions_baseline, average=None)
    print("metrics for analyte",j,'f1_scores, training set',f1_train,'f1_scores in testing set',f1_test)

Epoch 1/200
77/77 [==============================] - 5s 61ms/step - loss: 3.1326 - accuracy: 1.0000 - precision: 0.9757 - recall: 0.5810 - val_loss: 0.1788 - val_accuracy: 1.0000 - val_precision: 0.9750 - val_recall: 0.9611
Epoch 2/200
77/77 [==============================] - 4s 56ms/step - loss: 0.1562 - accuracy: 1.0000 - precision: 0.9741 - recall: 0.9693 - val_loss: 0.1289 - val_accuracy: 1.0000 - val_precision: 0.9749 - val_recall: 0.9726
Epoch 3/200
77/77 [==============================] - 4s 55ms/step - loss: 0.1014 - accuracy: 1.0000 - precision: 0.9744 - recall: 0.9743 - val_loss: 0.0655 - val_accuracy: 1.0000 - val_precision: 0.9785 - val_recall: 0.9778
Epoch 4/200
77/77 [==============================] - 4s 55ms/step - loss: 0.0370 - accuracy: 1.0000 - precision: 0.9889 - recall: 0.9889 - val_loss: 0.0206 - val_accuracy: 1.0000 - val_precision: 0.9955 - val_recall: 0.9951
Epoch 5/200
77/77 [==============================] - 4s 58ms/step - loss: 0.0129 - accuracy: 1.0000 - pr

Epoch 37/200
77/77 [==============================] - 4s 56ms/step - loss: 3.5530e-04 - accuracy: 1.0000 - precision: 0.9999 - recall: 0.9999 - val_loss: 0.0123 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 38/200
77/77 [==============================] - 4s 57ms/step - loss: 4.5102e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0132 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 39/200
77/77 [==============================] - 4s 58ms/step - loss: 6.4129e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0144 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 40/200
77/77 [==============================] - 4s 58ms/step - loss: 8.0481e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0132 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 41/200
77/77 [==============================] - 4s 57ms/step - loss: 4.8320e-0

Epoch 73/200
77/77 [==============================] - 4s 56ms/step - loss: 7.2454e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0197 - val_accuracy: 1.0000 - val_precision: 0.9979 - val_recall: 0.9979
Epoch 74/200
77/77 [==============================] - 4s 57ms/step - loss: 1.4515e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0197 - val_accuracy: 1.0000 - val_precision: 0.9986 - val_recall: 0.9986
Epoch 75/200
77/77 [==============================] - 4s 57ms/step - loss: 9.0459e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0199 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 76/200
77/77 [==============================] - 4s 55ms/step - loss: 1.9101e-04 - accuracy: 1.0000 - precision: 0.9999 - recall: 0.9999 - val_loss: 0.0210 - val_accuracy: 1.0000 - val_precision: 0.9979 - val_recall: 0.9979
Epoch 77/200
77/77 [==============================] - 4s 56ms/step - loss: 3.3106e-0

Epoch 109/200
77/77 [==============================] - 5s 59ms/step - loss: 1.9842e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0217 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 110/200
77/77 [==============================] - 4s 58ms/step - loss: 7.5150e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0186 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 111/200
77/77 [==============================] - 4s 58ms/step - loss: 3.8580e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0198 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 112/200
77/77 [==============================] - 4s 58ms/step - loss: 1.0132e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0222 - val_accuracy: 1.0000 - val_precision: 0.9983 - val_recall: 0.9983
Epoch 113/200
77/77 [==============================] - 4s 57ms/step - loss: 2.69

Epoch 180/200
77/77 [==============================] - 4s 57ms/step - loss: 1.3262e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0200 - val_accuracy: 1.0000 - val_precision: 0.9986 - val_recall: 0.9983
Epoch 181/200
77/77 [==============================] - 5s 59ms/step - loss: 1.2111e-04 - accuracy: 1.0000 - precision: 0.9999 - recall: 0.9999 - val_loss: 0.0285 - val_accuracy: 1.0000 - val_precision: 0.9979 - val_recall: 0.9979
Epoch 182/200
77/77 [==============================] - 4s 58ms/step - loss: 7.7029e-06 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0287 - val_accuracy: 1.0000 - val_precision: 0.9979 - val_recall: 0.9979
Epoch 183/200
77/77 [==============================] - 4s 58ms/step - loss: 2.0251e-05 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0250 - val_accuracy: 1.0000 - val_precision: 0.9986 - val_recall: 0.9983
Epoch 184/200
77/77 [==============================] - 5s 59ms/step - loss: 7.48

#### Analytes 13, 34 and 37 are below .800

### PCA before classification

In [18]:
from sklearn.decomposition import PCA
dfset3=b1.copy()
pca = PCA(n_components=30)
pca_result = pca.fit_transform(dfset3)

In [24]:
# Configuration options
feature_vector_length = 30
num_classes = 18000
X=pca_result
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2,random_state=0)
# Convert target classes to categorical ones
ytrain=Y_train
ytest=Y_test
Y_train = to_categorical(Y_train, num_classes)
Y_test = to_categorical(Y_test, num_classes)
print('Train dimension:')
print(X_train.shape)
print('Test dimension:')
print(X_test.shape)

Train dimension:
(14400, 50)
Test dimension:
(3600, 50)


In [25]:
# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')
# Create the model
model = Sequential()
#model.add(Dropout(0.2, input_shape=input_shape))
model.add(Dense(20, input_shape=input_shape, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Feature shape: (30,)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_10 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_11 (Dense)             (None, 18000)             378000    
Total params: 379,040
Trainable params: 379,040
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Configure the model and start training
import time
start_time = time.time()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
baseline_history=model.fit(X_train, Y_train, epochs=100, batch_size=64, verbose=1, validation_split=0.2)

Epoch 1/100


ValueError: in user code:

    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\eacun\anaconda38\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step

In [22]:
# Test the model after training
test_results = model.evaluate(X_test, Y_test, verbose=1)
print(test_results)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')
print("--- %s seconds ---" % (time.time() - start_time))

113/113 [==============================] - 1s 6ms/step - loss: 0.2208 - accuracy: 0.9417
[0.22077715396881104, 0.9416666626930237]
Test results - Loss: 0.22077715396881104 - Accuracy: 0.9416666626930237%
--- 223.01282048225403 seconds ---


In [23]:
#Computing F1-score
train_features = np.array(X_train)
test_features = np.array(X_test)
train_labels=np.array(Y_train)
test_labels=np.array(Y_test)
train_predictions_baseline = model.predict_classes(train_features, batch_size=150)
f1_train=sklearn.metrics.f1_score(ytrain, train_predictions_baseline, average="weighted")
test_predictions_baseline = model.predict_classes(test_features, batch_size=150)
f1_test=sklearn.metrics.f1_score(ytest, test_predictions_baseline, average="weighted")
print('f1_scores in training set',f1_train,'f1_scores in testing set',f1_test)

f1_scores in training set 0.9618240587439383 f1_scores in testing set 0.9419507751979359


In [24]:
num_classes = 18000
labels1=[1,2]
import time
start_time=time.time()
for j in labels1:
    yclass=y.copy()
    yclass[yclass!=j]=0 
    yclass[yclass==j]=1
    X_train, X_test, yclass_train, yclass_test = train_test_split(X,yclass,test_size=0.2,random_state=0)
    #print('Train dimension:');print(X_train.shape)
    #print('Test dimension:');print(X_test.shape)
    Y_train = to_categorical(yclass_train, num_classes)
    Y_test = to_categorical(yclass_test, num_classes)
    # Set the input shape
    input_shape = (feature_vector_length,) 
    #print(f'Feature shape: {input_shape}')
    # Create the model
    model = Sequential()
    model.add(Dropout(0.2, input_shape=input_shape))
    model.add(Dense(30, input_shape=input_shape, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    # Configure the model and start training
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.BinaryAccuracy(name="accuracy"), keras.metrics.Precision(name="precision"),keras.metrics.Recall(name="recall")])
    baseline_history=model.fit(X_train, Y_train, epochs=100, batch_size=100, validation_split=0.2)
    # Test the model after training
    test_results = model.evaluate(X_test, Y_test, verbose=1)
    #print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}% -Precision: {test_results[2]}% -Recall: {test_results[3]}%')
    #Computing F1-score
    train_features = np.array(X_train)
    test_features = np.array(X_test)
    train_labels=np.array(yclass_train)
    test_labels=np.array(yclass_test)
    train_predictions_baseline = model.predict_classes(train_features, batch_size=150)
    f1_train=sklearn.metrics.f1_score(train_labels, train_predictions_baseline, average=None)
    test_predictions_baseline = model.predict_classes(test_features, batch_size=100)
    f1_test=sklearn.metrics.f1_score(test_labels, test_predictions_baseline, average=None)
    print("metrics for analyte",j,'f1_scores, training set',f1_train,'f1_scores in testing set',f1_test)
print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/100
116/116 [==============================] - 5s 36ms/step - loss: 9.3857 - accuracy: 0.9999 - precision: 0.0342 - recall: 1.1892e-05 - val_loss: 3.0884 - val_accuracy: 0.9999 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
116/116 [==============================] - 4s 33ms/step - loss: 0.9189 - accuracy: 1.0000 - precision: 0.9784 - recall: 0.6275 - val_loss: 0.1238 - val_accuracy: 1.0000 - val_precision: 0.9729 - val_recall: 0.9729
Epoch 3/100
116/116 [==============================] - 4s 34ms/step - loss: 0.1104 - accuracy: 1.0000 - precision: 0.9761 - recall: 0.9761 - val_loss: 0.1108 - val_accuracy: 1.0000 - val_precision: 0.9729 - val_recall: 0.9729
Epoch 4/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0998 - accuracy: 1.0000 - precision: 0.9768 - recall: 0.9763 - val_loss: 0.1030 - val_accuracy: 1.0000 - val_precision: 0.9753 - val_recall: 0.9729
Epoch 5/100
116/116 [==============================] - 4s 34ms/step - loss: 0.0993 -

Epoch 37/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0022 - accuracy: 1.0000 - precision: 0.9997 - recall: 0.9997 - val_loss: 8.4650e-04 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 38/100
116/116 [==============================] - 4s 34ms/step - loss: 0.0026 - accuracy: 1.0000 - precision: 0.9988 - recall: 0.9988 - val_loss: 8.1121e-04 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 39/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0020 - accuracy: 1.0000 - precision: 0.9993 - recall: 0.9993 - val_loss: 8.1869e-04 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 40/100
116/116 [==============================] - 4s 32ms/step - loss: 0.0044 - accuracy: 1.0000 - precision: 0.9988 - recall: 0.9988 - val_loss: 8.0743e-04 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 41/100
116/116 [==============================] - 4s 33ms/step - loss:

116/116 [==============================] - 4s 35ms/step - loss: 0.0855 - accuracy: 1.0000 - precision: 0.9806 - recall: 0.9791 - val_loss: 0.0896 - val_accuracy: 1.0000 - val_precision: 0.9771 - val_recall: 0.9760
Epoch 7/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0847 - accuracy: 1.0000 - precision: 0.9800 - recall: 0.9783 - val_loss: 0.0860 - val_accuracy: 1.0000 - val_precision: 0.9771 - val_recall: 0.9760
Epoch 8/100
116/116 [==============================] - 4s 36ms/step - loss: 0.0903 - accuracy: 1.0000 - precision: 0.9770 - recall: 0.9755 - val_loss: 0.0831 - val_accuracy: 1.0000 - val_precision: 0.9771 - val_recall: 0.9760
Epoch 9/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0863 - accuracy: 1.0000 - precision: 0.9781 - recall: 0.9760 - val_loss: 0.0801 - val_accuracy: 1.0000 - val_precision: 0.9771 - val_recall: 0.9760
Epoch 10/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0783 - accuracy: 1.0000 - pre

116/116 [==============================] - 4s 33ms/step - loss: 0.0047 - accuracy: 1.0000 - precision: 0.9984 - recall: 0.9984 - val_loss: 0.0031 - val_accuracy: 1.0000 - val_precision: 0.9990 - val_recall: 0.9990
Epoch 79/100
116/116 [==============================] - 4s 34ms/step - loss: 0.0059 - accuracy: 1.0000 - precision: 0.9988 - recall: 0.9988 - val_loss: 0.0032 - val_accuracy: 1.0000 - val_precision: 0.9990 - val_recall: 0.9990
Epoch 80/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0073 - accuracy: 1.0000 - precision: 0.9979 - recall: 0.9979 - val_loss: 0.0032 - val_accuracy: 1.0000 - val_precision: 0.9990 - val_recall: 0.9990
Epoch 81/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0034 - accuracy: 1.0000 - precision: 0.9992 - recall: 0.9992 - val_loss: 0.0031 - val_accuracy: 1.0000 - val_precision: 0.9990 - val_recall: 0.9990
Epoch 82/100
116/116 [==============================] - 4s 33ms/step - loss: 0.0063 - accuracy: 1.0000 - 